# Final Project for INFO 6350
## Net ID: yz2685

In [1]:
# import our libraries
import requests
import urllib
from bs4 import BeautifulSoup
import simplejson as json
from urllib.request import Request, urlopen
from fake_useragent import UserAgent

In [2]:
# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
# content = requests.get(documents_url, headers=header)
# print(content)

# Grabbing the Financial Statements

In [3]:
# # create the list to hold the statement urls
# statements_url = []

# for report_dict in master_reports:
    
#     # define the statements we want to look for.
#     item1 = r"Consolidated Balance Sheets"
#     item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
#     item3 = r"Consolidated Statements of Cash Flows"
#     item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
#     # store them in a list.
#     report_list = [item1, item2, item3, item4]
    
#     # if the short name can be found in the report list.
#     if report_dict['name_short'] in report_list:
        
#         # print some info and store it in the statements url.
#         print('-'*100)
#         print(report_dict['name_short'])
#         print(report_dict['url'])
        
#         statements_url.append(report_dict['url'])

# Scraping the SEC Query Page

In [4]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'1318605', # Tesla
              # 'CIK':'320193', # Apple
              'type':'10-k',
              'dateb':'20230101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers=header)
# response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1318605&type=10-k&dateb=20230101&owner=exclude&start=&output=&count=100


In [5]:
doc_table = soup.find_all(class_ = "blueRow")
# print(doc_table)

In [6]:
data = soup.find_all(class_='blueRow')

list_10k = []

for i, row in enumerate(data): 
    for a in data[i].find_all('a', href=True):
        url = a['href']
        if (url.startswith('/Archives/edgar/')):
            list_10k.append("https://www.sec.gov"+ url)
            print("https://www.sec.gov"+ url)

https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/0001564590-21-004599-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459020004475/0001564590-20-004475-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/0001564590-18-002956-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/0001564590-16-013195-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/0001193125-14-069681-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/0001193125-12-137560-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847-index.htm


# Scraping Company Page

In [7]:
# get the links to the 10k reports 

def get10kPages(url):
    response = requests.get(url = url, headers=header)
    soup10k = BeautifulSoup(response.content, 'html.parser')
    # print(response)
    # print(response.url)
    return response.url

In [8]:
list_of_links = []
for link in list_10k:
    list_of_links.append(get10kPages(link))
print(list_of_links)

['https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/0001564590-21-004599-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459020004475/0001564590-20-004475-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/0001564590-18-002956-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/0001564590-16-013195-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/0001193125-14-069681-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/0001193125-12-137560-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847-index.htm']


In [9]:
# get the url for 10k report 
list_of_10ks = []

def get10kLinks(url):
    response = requests.get(url = url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    # for a in soup.find_all('a', href=True):
        # url = a['href']
        # print(url)
        
    suffix = "htm";

    tables = soup.find('table')
    rows = tables.find_all('tr')
    if len(rows) > 0:
        row10k = rows[1] # row 1 has link to 10k report
        # print(row10k)
        for a in row10k.find_all('a', href=True):
            url = a['href']
            if url.endswith(suffix):
                list_of_10ks.append("https://www.sec.gov"+ url)
                print("https://www.sec.gov"+ url)

In [10]:
for url in list_of_links:
    get10kLinks(url)

https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000095017022000796/tsla-20211231.htm
https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm
https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000156459020004475/tsla-10k_20191231.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/tsla-10k_20171231.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/tsla-10k_20151231.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/d668062d10k.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/d315279d10ka.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/d10k.htm


In [11]:
list_of_10ks

['https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000095017022000796/tsla-20211231.htm',
 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm',
 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000156459020004475/tsla-10k_20191231.htm',
 'https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/tsla-10k_20171231.htm',
 'https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/tsla-10k_20151231.htm',
 'https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/d668062d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/d315279d10ka.htm',
 'https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/d10k.htm']

In [12]:
# create a function that will scrap the 10k report 

from selenium import webdriver
# import chromedriver_binary  # Adds chromedriver binary to path
import time

data = []

driver = webdriver.Firefox()

def scrap10k(url):
    driver.get(url)

    time.sleep(5) # give browser some time to load the js 

    html = driver.page_source
    sp = BeautifulSoup(html)
    
    text = ""
    for d in sp.find_all(text=True):
        text += d.get_text()
    
    data.append(text)

In [13]:
for report in list_of_10ks:
    scrap10k(report)

In [14]:
len(data)

8

# Analysis

In [16]:
# import unicodedata

# text = unicodedata.normalize('NFKD', text.replace("\'", "'").replace("\ in\ form", " inform").replace("\n", " ").lower().strip())

In [ ]:
# print(text)
# dev_docs = [doc for doc in text.split('\n')] 

In [ ]:
# from   sklearn.feature_extraction.text import TfidfVectorizer
# from   nltk.corpus import stopwords
# import string

# # Making stopwords list
# stoplist = stopwords.words('english')
# for el in [i for i in string.punctuation]:
#     stoplist.append(el)

# # Set up vectorizer
# vectorizer = TfidfVectorizer(
#     encoding='utf-8',
#     min_df=1, # include words that occur in as few as a single document, 1 = a single document
#     max_df=1.0, # include words that occur in as many as all documents, 1.0 = 100% = all documents
#      binary=False,
#     norm='l2',
#     stop_words = stoplist,
#     use_idf=True
# )

# X = vectorizer.fit_transform(dev_docs)

In [ ]:
# print(X.shape)

In [ ]:
# print(X)